In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# The forward and backward passes

In [2]:
#export 
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train, y_train, x_valid, y_valid))

def normalize(x, m, s): return (x-m)/s

In [3]:
x_train, y_train, x_valid, y_valid = get_data()

In [4]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(0.1304), tensor(0.3073))

In [5]:
x_train = normalize(x_train, train_mean, train_std)
x_valid = normalize(x_valid, train_mean, train_std)

In [6]:
train_mean, train_std = x_train.mean(), x_train.std()
train_mean, train_std

(tensor(-7.6999e-06), tensor(1.))

In [7]:
#export 
def test_near_zero(a, tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [8]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [9]:
n, m = x_train.shape
c = y_train.max()+1
n, m, c

(50000, 784, tensor(10))

# Foundation version

In [10]:
nh = 50

In [11]:
# simplified kaiming init 
w1 = torch.randn(m, nh) / math.sqrt(m)
b1 = torch.zeros(nh)
w2 = torch.randn(nh, 1) / math.sqrt(nh)
b2 = torch.zeros(1)

In [12]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [13]:
x_valid.mean(), x_valid.std()

(tensor(-0.0059), tensor(0.9924))

In [14]:
def lin(x, w, b): return x@w + b

In [15]:
t = lin(x_valid, w1, b1)

In [16]:
t.mean(), t.std()

(tensor(-0.0925), tensor(0.9669))

In [17]:
def relu(x): return x.clamp_min(0.)

In [18]:
t = relu(lin(x_valid, w1, b1))

In [19]:
t.mean(), t.std()

(tensor(0.3370), tensor(0.5280))

# Delving deep into rectifiers
imagenet winning approach

In [20]:
# Pytorch uses kaiming init
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2/m)
w1.mean(), w1.std()

(tensor(0.0003), tensor(0.0502))

In [21]:
t = relu(lin(x_valid, w1, b1))
t.mean(), t.std()

(tensor(0.5622), tensor(0.8134))

In [22]:
#export
from torch.nn import init

In [23]:
w1 = torch.zeros(m, nh)
# Choosing ``'fan_in'`` preserves the magnitude of the variance of 
# the weights in the forward pass. 
# Choosing ``'fan_out'`` preserves the magnitudes in the backwards pass.
# divide by m or nh
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))

In [24]:
#init.kaiming_normal_??

In [25]:
#init.kaiming_uniform_??
# 3 * normal math.sqrt(3.0) = std

In [26]:
w1.mean(), w1.std()

(tensor(0.0005), tensor(0.0501))

In [27]:
# sqrt(3) 
1.73205080757* w1.std()

tensor(0.0867)

In [28]:
t.mean(), t.std()

(tensor(0.5212), tensor(0.8195))

In [29]:
w1.shape

torch.Size([784, 50])

In [30]:
import torch.nn

In [31]:
torch.nn.Linear(m, nh).weight.shape

torch.Size([50, 784])

In [32]:
#torch.nn.Linear.forward??

In [33]:
#torch.nn.functional.linear??

In [34]:
def relu(x):
    return x.clamp_min(0.) - 0.5

In [35]:
# kaiming init / he init for relu
w1 = torch.randn(m, nh)*math.sqrt(2./m)
t1 = relu(lin(x_valid, w1, b1))
t1.mean(), t1.std()

(tensor(0.0111), tensor(0.8029))

In [36]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [37]:
%timeit -n 10 _=model(x_valid)

4.22 ms ± 115 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [38]:
assert model(x_valid).shape == torch.Size([x_valid.shape[0], 1])

In [39]:
x_valid.shape

torch.Size([10000, 784])

# Loss: MSE

In [40]:
#export
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

In [41]:
y_train, y_valid = y_train.float(), y_valid.float()

In [42]:
preds = model(x_train)

In [43]:
preds.shape

torch.Size([50000, 1])

In [44]:
mse(preds, y_train)

tensor(29.5886)

# Gradient and backward pass

In [45]:
def mse_grad(inp, targ):
    # inp.shape = [50000, 1]
    # inp.squeeze().shape = [50000]
    # (inp.squeeze()-targ).unsqueeze(-1).shape = [50000, 1]
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0]

In [46]:
def relu_grad(inp, out):
    inp.g = (inp>0).float() * out.g

In [47]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    # [50000, 1] * [50000, 1]
    # .sum(0).shape = 1
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0)
    b.g = out.g.sum(0)

In [48]:
def forward_and_backward(inp, targ):
    # forward
    l1 = inp @ w1 + b1
    l2 = relu (l1)
    out = l2 @ w2 + b2
    #  dont need loss in backward
    loss = mse(out, targ)
    
    # backward pass
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [49]:
forward_and_backward(x_train, y_train)

In [50]:
# save for testing
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig = x_train.g.clone()

Use pytorch autograd to check gradients

In [51]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [52]:
def forward(inp, targ):
    # forward
    l1 = inp @ w12 + b12
    l2 = relu (l1)
    out = l2 @ w22 + b22
    #  dont need loss in backward
    return mse(out, targ)

In [53]:
loss = forward(xt2, y_train)

In [54]:
loss.backward()

In [55]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

# Refactor model

layers as classes

In [56]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self):
        self.inp.g = (self.inp>0).float()* self.out.g

In [57]:
class Lin():
    def __init__(self, w, b): self.w, self.b = w, b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [58]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze()-targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [59]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [60]:
w1.g, b1.g, w2.g, b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [61]:
%time loss = model(x_train, y_train)

CPU times: user 270 ms, sys: 11.5 ms, total: 282 ms
Wall time: 71.7 ms


In [62]:
%time model.backward()

CPU times: user 4.74 s, sys: 2.12 s, total: 6.86 s
Wall time: 1.72 s


In [63]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

Module.forward()

don't need duplicat code for self.inp = inp return self.out

# Important part

In [64]:
class Module():
    def __call__(self, *args):
        self.args = args
        # call forward function
        self.out = self.forward(*args)
        return self.out
    
    def forward(self):
        # the class doenst define forward, raise error
        raise Exception('not implemented')
        
    def backward(self):
        self.bwd(self.out, *self.args)

In [65]:
class Relu(Module):
    def forward(self, inp):
        return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp):
        inp.g = (inp>0).float() * out.g

In [66]:
class Lin(Module):
    def __init__(self, w, b):
        self.w, self.b = w,b
    
    def forward(self, inp): 
        return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi, bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

In [67]:
class Mse(Module):
    def forward(self, inp, targ): 
        return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): 
        inp.g = 2*(inp.squeeze() - targ).unsqueeze(-1) / targ.shape[0]

In [68]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1, b1), Relu(), Lin(w2, b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [69]:
w1.g, b1.g, w2.g, b2.g = [None]*4
model = Model()

In [70]:
%time loss = model(x_train, y_train)

CPU times: user 285 ms, sys: 1.2 ms, total: 286 ms
Wall time: 73.6 ms


In [71]:
%time model.backward()

CPU times: user 179 ms, sys: 77 ms, total: 256 ms
Wall time: 64.5 ms


In [72]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

# Without Einsum

In [73]:
class Lin(Module):
    def __init__(self, w, b):
        self.w, self.b = w, b
        
    def forward(self, inp): 
        return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)

In [74]:
w1.g, b1.g, w2.g, b2.g = [None]*4
model = Model()

In [75]:
%time loss = model(x_train, y_train)

CPU times: user 87 ms, sys: 2.17 ms, total: 89.2 ms
Wall time: 22.3 ms


In [76]:
%time model.backward()

CPU times: user 187 ms, sys: 41 ms, total: 228 ms
Wall time: 57.5 ms


In [77]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

# nn.Linear and nn.Module

In [79]:
#export 
from torch import nn

In [82]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in, nh), nn.ReLU(), nn.Linear(nh, n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x=l(x)
        return self.loss(x.squeeze(), targ)

In [83]:
model = Model(m, nh, 1)

In [86]:
%time loss = model(x_train, y_train)

CPU times: user 76.6 ms, sys: 0 ns, total: 76.6 ms
Wall time: 19.3 ms


In [87]:
%time loss.backward()

CPU times: user 220 ms, sys: 28.1 ms, total: 248 ms
Wall time: 65.1 ms


# Export

In [89]:
!./notebook2script.py 02_fully_connected.ipynb

Converted 02_fully_connected.ipynb to exp/nb_02.py


In [ ]:
# chmod -R 700 ~/Documents